In [1]:
exp_str = 'landau-zener m-sweep'

from qiskit import IBMQ #, transpile #, schedule, pulse
# from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister, ClassicalRegister
# from qiskit.opflow import Y, Z, PauliTrotterEvolution, Suzuki
# from qiskit_research import utils

import numpy as np

# import matplotlib.pyplot as plt
# plt.style.use('dark_background')
# plt.rcParams['figure.figsize'] = [5, 5]

In [2]:
IBMQ.load_account()
try:
    provider = IBMQ.get_provider(hub='ibm-q-internal', group='mission-critical', project='bronn')
    backend = provider.get_backend('ibm_lagos')
except:
    provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')
    backend = provider.get_backend('ibmq_bogota') # checking gate directions

In [3]:
job_ids = ['62acb5bef09eced714e38b2c'] # 30 Trotter steps!

In [4]:
job = backend.retrieve_job(job_ids[-1])
result = job.result()
metadata = result.header
num_shots = result.results[0].shots
counts = result.get_counts()

In [5]:
metadata.type

'm-sweep'

In [6]:
if metadata.type == 'm-sweep':
    m_range = np.array(metadata.m).round(4)
    w_range = np.array(metadata.ω).round(4)
    g_set = metadata.g
    c_set = metadata.c
    t_set = metadata.t
    dt_set = metadata.dt
    Nt = metadata.trotter

In [7]:
Zexp_m_w = []
for midx,_ in enumerate(m_range):
    Zexp_w = []
    for widx,_ in enumerate(w_range):
        count = counts[widx + midx*len(w_range)]
        if '1' in count:
            Zexp_w.append((1 - 2*count['1']/num_shots))
        else:
            Zexp_w.append(1)
            
    Zexp_m_w.append(Zexp_w)

In [15]:
if metadata.type == 'm-sweep':
    for m_analyze in m_range:
        midx = int(np.where(np.round(np.asarray(m_range), 2) == m_analyze)[0])
        idx0 = midx*len(w_range) # starting index
        w0 = w_range[0]
        dw = round(w_range[1] - w_range[0], 2)

        param_decoder = ['g='+str(round(g_set, 2))+', m=' + str(round(m_analyze, 2)) + 
                                  ', w=' + str(round(w0, 2))]
    
        np.save('Data/landau-zener/c0p1_g1p5/N30/SE_1trot_N_2_c_'+str(c_set)+'_dt_'+str(dt_set)+'_t_'+str(t_set)+'_w0_'+str(w0)+'_dw_'+str(dw)+'_m_'+str(m_analyze)+'_g_'+str(g_set), Zexp_m_w)
        np.save('Data/landau-zener/c0p1_g1p5/N30/w_N_2_c_'+str(c_set)+'_dt_'+str(dt_set)+'_t_'+str(t_set)+'_w0_'+str(w0)+'_dw_'+str(dw)+'_m_'+str(m_analyze)+'_g_'+str(g_set), w_range)
        np.save('Data/landau-zener/c0p1_g1p5/N30/decoder_N_2_c_'+str(c_set)+'_dt_'+str(dt_set)+'_t_'+str(t_set)+'_w0_'+str(w0)+'_dw_'+str(dw)+'_m_'+str(m_analyze)+'_g_'+str(g_set), param_decoder)


In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table